## 2019

In [18]:
import pandas as pd
import numpy as np

df2019 = pd.read_excel('./data/2019.xlsx', sheet_name= 'Sheet1', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=55)
df2019.columns = ['팀 코드', '주제명', '팀원1', '팀원2', '팀원3']

df2019.head()

,팀 코드,주제명,팀원1,팀원2,팀원3
0,가-1,어떻게 해수욕장 인구를 효율적으로 측정할까?,2115 박장현,2117 송지원,NaN
1,가-2,"해수욕장, 언제 갈래요?",2401 곽서현,2402 박지윤,NaN
2,가-3,이안류 위험도를 예측하여 대비할 수 있을까?,2403 이영진,2503 박소예,NaN
3,가-4,어떻게 낚시 이용자를 증대시키고 해얀지역 경제를 활성화시킬 수 있을까?,2506 진혜민,2510 노윤호,NaN
4,가-5,해수욕장을 100%로 즐기려면?,2208 김도훈,2314 이정홍,NaN


In [19]:
df2019['팀 코드'] = df2019['팀 코드'].str.replace('가', 'A')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('나', 'B')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('다', 'C')
df2019['팀 코드'] = df2019['팀 코드'].str.replace('라', 'D')

df2019['팀 코드'] = df2019['팀 코드'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))

df2019['팀원1학번'], df2019['팀원1이름'] = df2019['팀원1'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원1'].apply(lambda x: x[4:] if x is not np.nan else x)

df2019['팀원2학번'], df2019['팀원2이름'] = df2019['팀원2'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원2'].apply(lambda x: x[4:] if x is not np.nan else x)

df2019['팀원3학번'], df2019['팀원3이름'] = df2019['팀원3'].apply(lambda x: x[:4] if x is not np.nan else x), df2019['팀원3'].apply(lambda x: x[4:] if x is not np.nan else x)
df2019.drop(['팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df2019.tail()

,팀 코드,주제명,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
48,D08,어선의 최적항로는 어떤 모양일까?,3408,김장현,3409,김재영,NaN,NaN
49,D09,소형 배로 적조 문제를 해결하고 해양데이터까지 수집하면 어떨까?,3416,서영국,3504,공명준,NaN,NaN
50,D10,뱃멀미를 어떻게 예방할 수 있을까?,1110,배시우,1216,이상민,1518,조성현
51,D11,해류와 암초의 영향을 고려한 선박의 최적 경로는?,1108,박정원,1109,박주영,1415,이유신
52,D12,최적의 선박 항로는? 골든 타임 확보를 위한 선박의 안전성 보장,1406,김원준,1419,최강훈,NaN,NaN


In [20]:
properties = ['팀 코드', '주제명', '팀원1 학번', '팀원1 이름', '팀원2 학번', '팀원2 이름', '팀원3 학번', '팀원3 이름']

def get_student_number(student_number):
  try:
    return str(int(student_number))
  except:
    return '0000'

def set_metadata(team_info, document, year = 2019):
  document.metadata = {}

  document.metadata['Year'] = year
  document.metadata['Team code'] = team_info['팀 코드']
  document.metadata['Title'] = team_info['주제명']

  try:
    document.metadata['Team name'] = team_info['팀명']
  except:
    document.metadata['Team name'] = None

  document.metadata['Teammate #1 name'] = team_info['팀원1이름']
  document.metadata['Teammate #1 number'] = get_student_number(team_info['팀원1학번'])
  document.metadata['Teammate #2 name'] = team_info['팀원2이름']
  document.metadata['Teammate #2 number'] = get_student_number(team_info['팀원2학번'])
  document.metadata['Teammate #3 name'] = team_info['팀원3이름']
  document.metadata['Teammate #3 number'] = get_student_number(team_info['팀원3학번'])
  try:
    document.metadata['Youtube link'] = team_info['YOUTUBE 영상 주소']
  except:
    document.metadata['Youtube link'] = None

  return document


In [21]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 학번 추출

PDF_DIR = 'OceanICT_Data/2019_OceanICT/포스터/'
files = os.listdir(PDF_DIR)
file_list = sorted(files, key = lambda x: int(p.match(x).group()))

split_documents19 = []

for f in file_list:
  loader = PyMuPDFLoader(PDF_DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df2019.loc[df2019[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  try:
    new_document = set_metadata(result.iloc[0], document)
    split_documents19.append(new_document)
  except:
    continue

In [22]:
print(len(split_documents19))

54


In [23]:
from pprint import pprint

type(split_documents19[0])

langchain_core.documents.base.Document

## 2020

In [24]:
df20 = pd.read_excel('./data/2020.xlsx', sheet_name= '팀 코드 정리', usecols=[i + 8 for i in range(7)], skiprows=3, nrows=72)
df20.drop(['유형'], axis=1, inplace=True)
df20 = df20.rename(columns={'연구 제목': '주제명'})
df20.tail()

,팀원1,팀원2,팀원3,주제명,팀 번호,YOUTUBE 영상 주소
66,2406 김근형,2409 김준우,NaN,효율적인 등대 배치를 위한 통계적인 최적화 방법에 대한 탐구,D-8,https://youtu.be/yXW9bSgih-Y
67,2407 김민성,2412 류운비,NaN,연안에서 최적의 선박 회피경로 구하기,D-9,https://youtu.be/7qKrCrHhFQc
68,2520 최호영,NaN,NaN,선박 간 정보공유 플랫폼 구축을 통한 어획의 편의성 증대,D-10,https://youtu.be/SB1BPdxQ-h4
69,3214 정상,3314 한경찬,NaN,선박에서 코로나바이러스 피해를 줄일 수 있을까?,D-11,https://youtu.be/sV8qrHp3vcs
70,3307 박관우,3407 박성현,NaN,선박 사고 시 가장 효율적인 대처 방안 모색,D-12,https://youtu.be/-tG4NkGq__4


In [25]:
df20['팀 코드'] = df20['팀 번호'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))

for i in range(1, 4):
    df20[f'팀원{i}학번'], df20[f'팀원{i}이름'] = df20[f'팀원{i}'].apply(lambda x: x[:4] if x is not np.nan else x), df20[f'팀원{i}'].apply(lambda x: x[5:] if x is not np.nan else x)

df20.drop(['팀 번호', '팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df20.head()

,주제명,YOUTUBE 영상 주소,팀 코드,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
0,해운대 해수욕장 파속측정을 통한 이안류 발생시 알림 수신 프로그래밍,https://www.youtube.com/watch?v=JSnm4dtmU5c,A01,1118,이현승,1115,이영진,1308,김정현
1,해양데이터를 이용해 이안류 피해를 줄일 수 있을까?,https://www.youtube.com/watch?v=Y5YvaB7awd4,A02,1210,안동준,1212,이건희,1215,장연수
2,해양 양식장 관리 시스템 설계,https://youtu.be/En4Bn1UEuyw,A03,1216,정민서,1418,장문정,1515,이승헌
3,수온에 따른 어획량예측,https://youtu.be/eNqs5N7W9v8,A04,1311,박정윤,1312,백재승,1320,정윤상
4,해수욕장 인명사고 예방을 위한 경보 시스템 제작,https://www.youtube.com/watch?v=m4bCSDfgRaA,A05,2103,안주연,2104,이수민,NaN,NaN


In [26]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import tqdm

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 팀코드 추출

DIR = 'OceanICT_Data/2020_OceanICT/'
file_list = os.listdir(DIR)
file_list_pdf = [file for file in file_list if file.endswith(".pdf")]
file_list_pptx = [file for file in file_list if file.endswith(".pptx")]


split_documents20 = []

for f in file_list_pdf:
  loader = PyMuPDFLoader(DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df20.loc[df20[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  new_document = set_metadata(result.iloc[0], document, 2020)
  split_documents20.append(new_document)

for f in tqdm.tqdm(file_list_pptx):
  loader = UnstructuredPowerPointLoader(DIR + f)
  document = loader.load()[0]
  number = p.findall(f)[1]

  for i in range(1,4):
    result = df20.loc[df20[f'팀원{i}학번'] == number]
    if not result.empty:
      break

  new_document = set_metadata(result.iloc[0], document, 2020)
  split_documents20.append(new_document)

100%|██████████| 70/70 [00:03<00:00, 20.43it/s]


In [27]:
len(split_documents20)

74

## 2021

In [28]:
df21_A = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 문화와 관광 진흥', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=20)
df21_B = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 생태계 및 환경 보존', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=30)
df21_C = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 자원의 이용 기반 구축', usecols=[i + 2 for i in range(5)], skiprows=2, nrows=34)
df21_D = pd.read_excel('./data/2021.xlsx', sheet_name= '해양 선박 관련 기술', usecols=[i + 2 for i in range(5)], skiprows=1, nrows=10)

df21 = pd.concat([df21_A, df21_B, df21_C, df21_D])
df21 = df21.rename(columns={'주제': '주제명'})
df21.head()

,연번,주제명,팀원1,팀원2,팀원3
0,A-1,해변 관광환경 개선 및 만족도 향상을 위한 해변 추천 서비스 제작하기,1413 신재현,1420 하승훈,NaN
1,A-2,해양 정보를 이용한 플레이리스트 추천 프로그램,2104 윤현서,2105 이다은,NaN
2,A-3,인공지능과 오픈 API를 이용한 해양 경보 어플리케이션 제작,2108 김경태,2119 최준영,NaN
3,A-4,부산 수산시장의 해산물 가격 판단 서비스,2213 안동준,2215 장연수,NaN
4,A-5,역기압 효과를 이용한 파고 높이 계산을 통해 어떻게 풍랑주의보 알리미 제작할 수 있을까?,2217 정재민,2218 조현준,NaN


In [29]:
df21['팀 코드'] = df21['연번'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))
df21['연번'] = df21['연번'].str.replace('-', '')

for i in range(1, 4):
    df21[f'팀원{i}학번'], df21[f'팀원{i}이름'] = df21[f'팀원{i}'].apply(lambda x: x[:4] if x is not np.nan else x), df21[f'팀원{i}'].apply(lambda x: x[5:] if x is not np.nan else x)

df21.drop(['팀원1', '팀원2', '팀원3'], axis=1, inplace= True)

df21.head()

,연번,주제명,팀 코드,팀원1학번,팀원1이름,팀원2학번,팀원2이름,팀원3학번,팀원3이름
0,A1,해변 관광환경 개선 및 만족도 향상을 위한 해변 추천 서비스 제작하기,A01,1413,신재현,1420,하승훈,NaN,NaN
1,A2,해양 정보를 이용한 플레이리스트 추천 프로그램,A02,2104,윤현서,2105,이다은,NaN,NaN
2,A3,인공지능과 오픈 API를 이용한 해양 경보 어플리케이션 제작,A03,2108,김경태,2119,최준영,NaN,NaN
3,A4,부산 수산시장의 해산물 가격 판단 서비스,A04,2213,안동준,2215,장연수,NaN,NaN
4,A5,역기압 효과를 이용한 파고 높이 계산을 통해 어떻게 풍랑주의보 알리미 제작할 수 있을까?,A05,2217,정재민,2218,조현준,NaN,NaN


In [30]:
import os
from langchain_community.document_loaders import PyMuPDFLoader

import re	# 정규식 라이브러리

PDF_DIR = 'OceanICT_Data/2021_OceanICT/포스터/'
file_list = os.listdir(PDF_DIR)

split_documents21 = []
a= []

for f in file_list:
  loader = PyMuPDFLoader(PDF_DIR + f)
  try:
    document = loader.load()[0]
  except:
    continue
  result = df21.loc[df21[f'연번'] == f[:-4]]

  new_document = set_metadata(result.iloc[0], document, year = 2021)
  a.append(result.iloc[0])
  split_documents21.append(new_document)

In [31]:
len(split_documents21)

86

## 2024

In [97]:
df24 = pd.read_excel('./data/2024.xlsx', sheet_name= '팀별', usecols=[i + 1 for i in range(9)], skiprows=2, nrows=98)
df24 = df24.rename(columns={'섹션\n번호': '팀 코드'})
df24.drop(0, inplace=True)
df24.drop(['학년'], axis= 1,inplace=True)

df24['팀 코드'] = df24['팀 코드'].apply(lambda x:x.split('-')[0] + x.split('-')[1].zfill(2))
df24.fillna('0000', inplace=True)

for i in range(1, 4):
    try:
        df24[f'팀원{i}학번'] = df24[f'팀원{i} 학번'].astype(int).astype(str)
        df24.rename(columns={f'팀원{i} 이름': f'팀원{i}이름'},inplace=True)
        df24.drop([f'팀원{i} 학번'],inplace=True,axis=1)
    except:
        continue

df24['주제명'] = df24['팀명']
df24['얘 들어갔어요'] = np.zeros(df24.shape[0],dtype=int)

df24.head()

,팀 코드,팀원1이름,팀원2이름,팀원3이름,팀명,팀원1학번,팀원2학번,팀원3학번,주제명,얘 들어갔어요
1,B01,0000,0000,0000,클로렐라쌀밥,1102,1306,0,클로렐라쌀밥,0
2,B02,0000,0000,0000,유령인간,1103,1104,0,유령인간,0
3,B03,0000,0000,0000,디버깅 마스터,1105,1106,0,디버깅 마스터,0
4,A01,0000,0000,0000,정보통신기술,1110,1112,0,정보통신기술,0
5,B04,0000,0000,0000,별미,1111,1505,0,별미,0


In [120]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import tqdm

import re	# 정규식 라이브러리
p = re.compile('\d+') # 괄호 안에 있는 팀코드 추출

DIR = 'OceanICT_Data/2024_OceanICT/'
file_list = os.listdir(DIR)

split_documents24 = []


for f in file_list:
    files = os.listdir(DIR + f)
    for i in files:
        if i.endswith('.pdf'):
            loader = PyMuPDFLoader(DIR + f +'/' + i)
        
        elif i.endswith(('.pptx', '.ppt')):
            loader = UnstructuredPowerPointLoader(DIR + f +'/' + i)
        
        else: continue

        number = p.findall(i)
        print(i, number)

        document = loader.load()[0]
        try:
            number[0] = ''
        except: continue

        for j in number:
            result = df24.loc[df24[f'팀원1학번'] == j]
            if not result.empty:
                df24.loc[df24[f'팀원1학번'] == j, '얘 들어갔어요'] = 1
                break
        
        try:
            new_document = set_metadata(result.iloc[0], document, year = 2024)
            split_documents24.append(new_document)
        except:
            continue

파도위의혁명단_2509김민규_2511김준민.pdf ['2509', '2511']
파도위의혁명단_2509 김민규_2511 김준민.pptx ['2509', '2511']
부희주들_2304정유진_2305하은결.pptx ['2304', '2305']
빈포인트_2414유예준_2416임성훈.pptx ['2414', '2416']
빈포인트_2414유예준_2416임성훈.pdf ['2414', '2416']
잠수함_1405장윤_1502김소원_1503박지유.pptx ['1405', '1502', '1503']
아두이노를 활용한 트로마츠 웨이브 최적화(2104 정여진, 2118 이재호).pptx ['2104', '2118']
부산과학고의 질병관리청_2501김민채_2502김윤정.pdf ['2501', '2502']
부산과학고의 질병관리청_2501김민채_2502김윤정.pptx ['2501', '2502']
재!활용로켓의대가나일롱마스크_2310김태규_2313박예준.pdf ['2310', '2313']
재!활용로켓의대가나일롱마스크_2310김태규_2313박예준.pptx ['2310', '2313']
펩시보다는 코카콜라_3213황성빈_3310유상현 복사본.pptx ['3213', '3310']
오션 포스터 양식.pdf []
3학년3반_3303양수민_3308박새빈_3313정성원.pptx ['3', '3', '3303', '3308', '3313']
3학년3반_3303양수민_3308박새빈_3313정성원.pdf ['3', '3', '3303', '3308', '3313']
오일팝퀴즈_2402서해원_2403손지민.pdf ['2402', '2403']
도라에몽_3307김지운_3311이용욱.pptx ['3307', '3311']
금쪽이 리턴즈_3101권지민_3105김민수_3203이지원.pptx ['3101', '3105', '3203']
SUMPM_2214이동윤_2218최현민.pdf ['2214', '2218']
SUMPM_2214이동윤_2218최현민.pptx ['2214', '2218']
(B53)

In [110]:
df24.loc[df24[f'팀원1학번'] == "1215"]

,팀 코드,팀원1이름,팀원2이름,팀원3이름,팀명,팀원1학번,팀원2학번,팀원3학번,주제명,얘 들어갔어요
14,C02,0000,0000,0000,부산갈맷길,1215,1217,0,부산갈맷길,1


In [114]:
pd.set_option('display.max_rows', None)
df24

,팀 코드,팀원1이름,팀원2이름,팀원3이름,팀명,팀원1학번,팀원2학번,팀원3학번,주제명,얘 들어갔어요
1,B01,0000,0000,0000,클로렐라쌀밥,1102,1306,0,클로렐라쌀밥,1
2,B02,0000,0000,0000,유령인간,1103,1104,0,유령인간,1
3,B03,0000,0000,0000,디버깅 마스터,1105,1106,0,디버깅 마스터,1
4,A01,0000,0000,0000,정보통신기술,1110,1112,0,정보통신기술,0
5,B04,0000,0000,0000,별미,1111,1505,0,별미,0
6,B05,0000,0000,0000,오진우뭐먹어나도줘,1113,1116,0,오진우뭐먹어나도줘,0
7,B06,0000,0000,0000,바다의 왕자,1114,1118,0,바다의 왕자,1
8,D01,0000,0000,0000,바다와 고딩,1117,1409,0,바다와 고딩,1
9,B07,0000,0000,0000,SeaPerature,1203,1214,0,SeaPerature,1
10,B08,0000,0000,0000,Radioactive BSS,1205,1211,0,Radioactive BSS,1


In [94]:
print(len(split_documents24))

118
